# Pattern Discovery MP1

In [160]:
import pandas as pd
import numpy as np
from itertools import combinations
from collections import Counter

## Read

In [3]:
categories = pd.read_table('categories.txt', header=None, names=['category'])
categories.shape

(77185, 1)

## Apriori
with join

In [164]:
## 1-len item
# set minimum support
sup = 0.01
n = categories.shape[0]
min_sup = np.floor(n*sup)

# join: category data to list of categories and count items
categories_list = categories.category.str.split(";")
count = Counter(category for category_list in categories_list for category in category_list)

# scan: for minimum counts
scan = dict()
[scan.update({key: value}) for key, value in count.items() if value > min_sup]
print('scan done')

scan done


In [174]:
## 2-len items
# self- join and count
join = list(combinations(scan.keys(), 2))
count = Counter(tuple(sorted(comb)) for category_list in categories_list for comb in combinations(category_list, 2))

# scan
scan2 = dict()
[scan2.update({key: value}) for key, value in count.items() for item in join if set(key) == set(item) and value > min_sup]

{('Active Life', 'Fitness & Instruction'): 1442,
 ('American (New)', 'Restaurants'): 1593,
 ('American (Traditional)', 'Restaurants'): 2416,
 ('Auto Repair', 'Automotive'): 1716,
 ('Bakeries', 'Food'): 1115,
 ('Bars', 'Nightlife'): 4328,
 ('Bars', 'Pubs'): 874,
 ('Bars', 'Restaurants'): 2423,
 ('Bars', 'Sports Bars'): 818,
 ('Beauty & Spas', 'Hair Salons'): 2091,
 ('Beauty & Spas', 'Nail Salons'): 1667,
 ('Breakfast & Brunch', 'Restaurants'): 1369,
 ('Burgers', 'Fast Food'): 774,
 ('Burgers', 'Restaurants'): 1774,
 ('Cafes', 'Restaurants'): 1002,
 ('Chinese', 'Restaurants'): 1629,
 ('Coffee & Tea', 'Food'): 2199,
 ('Dentists', 'General Dentistry'): 823,
 ('Dentists', 'Health & Medical'): 1195,
 ('Doctors', 'Health & Medical'): 1694,
 ('Event Planning & Services', 'Hotels'): 1431,
 ('Event Planning & Services', 'Hotels & Travel'): 1471,
 ('Fashion', 'Shopping'): 3078,
 ('Fashion', "Women's Clothing"): 1138,
 ('Fast Food', 'Restaurants'): 2851,
 ('Food', 'Grocery'): 1424,
 ('Food', 'Ice 

In [175]:
## 3-len items
k = 3
# self-join(k-2 match)
len(scan2.keys())
join = [set(key).union(set(key2))  for key in scan2.keys() for key2 in scan2.keys() if len(set(key).intersection(set(key2)))>= k-2 and key !=key2 ]
join = np.unique(join)
# count
count = Counter(tuple(sorted(comb)) for category_list in categories_list for comb in  combinations(category_list, k))

# scan
scan3 = dict()
[scan3.update({key: value}) for key, value in count.items() for item in join if set(key) == item and value>min_sup]
scan3

{('Bars', 'Nightlife', 'Pubs'): 874,
 ('Bars', 'Nightlife', 'Restaurants'): 2423,
 ('Bars', 'Nightlife', 'Sports Bars'): 818,
 ('Burgers', 'Fast Food', 'Restaurants'): 774,
 ('Dentists', 'General Dentistry', 'Health & Medical'): 823,
 ('Event Planning & Services', 'Hotels', 'Hotels & Travel'): 1431,
 ('Fashion', 'Shopping', "Women's Clothing"): 1138}

In [176]:
# combine scan dictionary
scan.update(scan2)
scan.update(scan3)

In [177]:
# save file
with open('patterns2.txt', 'w') as file:
    for key, value in scan.items():
        if type(key) is str:
            file.write(str(value)+":"+key+'\n')
        else:
            file.write(str(value)+':'+';'.join(key)+'\n')

## Apriori
without join

In [178]:
def apriori(data, sup = 0.01):
    # set minimum support
    n = data.shape[0]
    min_sup = np.floor(n*sup)

    # to list of sets
    categories_list = [set(category_list) for category_list in data.category.str.split(';')]
    
    # result container
    rules = dict()
    
    i = 1
    while(True):
        # get join of categories_list
        # sort in alphabetic order
        join = Counter(tuple(sorted(comb)) for category_list in categories_list for comb in combinations(category_list, i))
        [rules.update({key: value}) for key, value in join.items() if value > min_sup]
        i += 1
        if all([value < min_sup for value in list(join.values())]):
            break
    return rules

In [180]:
%%time
rules = apriori(categories)

{('Active Life',): 3103,
 ('Active Life', 'Fitness & Instruction'): 1442,
 ('American (New)',): 1593,
 ('American (New)', 'Restaurants'): 1593,
 ('American (Traditional)',): 2416,
 ('American (Traditional)', 'Restaurants'): 2416,
 ('Arts & Entertainment',): 2271,
 ('Auto Repair',): 1716,
 ('Auto Repair', 'Automotive'): 1716,
 ('Automotive',): 4208,
 ('Bakeries',): 1115,
 ('Bakeries', 'Food'): 1115,
 ('Bars',): 4328,
 ('Bars', 'Nightlife'): 4328,
 ('Bars', 'Nightlife', 'Pubs'): 874,
 ('Bars', 'Nightlife', 'Restaurants'): 2423,
 ('Bars', 'Nightlife', 'Sports Bars'): 818,
 ('Bars', 'Pubs'): 874,
 ('Bars', 'Restaurants'): 2423,
 ('Bars', 'Sports Bars'): 818,
 ('Beauty & Spas',): 6583,
 ('Beauty & Spas', 'Hair Salons'): 2091,
 ('Beauty & Spas', 'Nail Salons'): 1667,
 ('Breakfast & Brunch',): 1369,
 ('Breakfast & Brunch', 'Restaurants'): 1369,
 ('Burgers',): 1774,
 ('Burgers', 'Fast Food'): 774,
 ('Burgers', 'Fast Food', 'Restaurants'): 774,
 ('Burgers', 'Restaurants'): 1774,
 ('Cafes',): 10

In [ ]:
# Part 1
keys = list(zip(*rules.items()))[0]
keys = [key for key in keys if len(key)==1]

one_rules = dict()
for key in keys:
    one_rules.update({key: rules.get(key)})

In [ ]:
# save
with open('patterns1.txt', 'w') as file:
    for key, value in one_rules.items():
        file.write(str(value) + ":" + str(key[0])+'\n')

In [181]:
# save Part 2
with open('patterns.txt', 'w') as file:
    for key, value in rules.items():
        file.write(str(value)+':'+';'.join(key) +'\n')